In [1]:
import time
import numpy as np
import tensorflow as tf

D:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [17]:
import reader

In [5]:
url = 'http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz'
import wget
wget.download(url, 'data/')

100% [........................................................................] 34869662 / 34869662

'data//simple-examples.tgz'

In [25]:
init_scale = 0.1
learning_rate = 1.0
max_grad_norm = 5
num_layers = 2
num_steps = 20
hidden_size_1 = 256
hidden_size_2 = 128
max_epoch_decay_lr = 4
max_epoch = 15
keep_prob = 1
decay = 0.5
batch_size = int(60)
vocab_size = 10000
embedding_vector_size = 200
is_training = 1
data_dir = 'data/simple-examples/data'

In [15]:
sess = tf.InteractiveSession()

In [18]:
raw_data = reader.ptb_raw_data(data_dir)
train_data, valid_data, test_data, vocab, word_to_id = raw_data

In [20]:
len(train_data)

929589

In [21]:
def id_to_word(id_list):
    line = []
    for w in id_list:
        for word,wid in word_to_id.items():
            if wid ==w:
                line.append(word)
    return line
print(id_to_word(train_data[0:100]))

['aer', 'banknote', 'berlitz', 'calloway', 'centrust', 'cluett', 'fromstein', 'gitano', 'guterman', 'hydro-quebec', 'ipo', 'kia', 'memotec', 'mlx', 'nahb', 'punts', 'rake', 'regatta', 'rubens', 'sim', 'snack-food', 'ssangyong', 'swapo', 'wachter', '<eos>', 'pierre', '<unk>', 'N', 'years', 'old', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'nov.', 'N', '<eos>', 'mr.', '<unk>', 'is', 'chairman', 'of', '<unk>', 'n.v.', 'the', 'dutch', 'publishing', 'group', '<eos>', 'rudolph', '<unk>', 'N', 'years', 'old', 'and', 'former', 'chairman', 'of', 'consolidated', 'gold', 'fields', 'plc', 'was', 'named', 'a', 'nonexecutive', 'director', 'of', 'this', 'british', 'industrial', 'conglomerate', '<eos>', 'a', 'form', 'of', 'asbestos', 'once', 'used', 'to', 'make', 'kent', 'cigarette', 'filters', 'has', 'caused', 'a', 'high', 'percentage', 'of', 'cancer', 'deaths', 'among', 'a', 'group', 'of']


In [24]:
type(batch_size)

float

In [26]:
itera = reader.ptb_iterator(train_data, batch_size, num_steps)
first_touple = itera.__next__()
x = first_touple[0]
y = first_touple[1]
x.shape

(60, 20)

In [27]:
input_data = tf.placeholder(tf.int32, [batch_size, num_steps])
target = tf.placeholder(tf.int32, [batch_size, num_steps])
feed_dict = {input_data: x, target: y}

In [29]:
lstm_l1 = tf.contrib.rnn.BasicLSTMCell(hidden_size_1, forget_bias = 0.0)
lstm_l2 = tf.contrib.rnn.BasicLSTMCell(hidden_size_2, forget_bias = 0.0)
stacked_lstm = tf.contrib.rnn.MultiRNNCell([lstm_l1,lstm_l2])

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


In [30]:
init_state = stacked_lstm.zero_state( batch_size, tf.float32)
init_state

(LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState/zeros:0' shape=(60, 256) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState/zeros_1:0' shape=(60, 256) dtype=float32>),
 LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState_1/zeros:0' shape=(60, 128) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState_1/zeros_1:0' shape=(60, 128) dtype=float32>))

In [31]:
sess.run(init_state, feed_dict)

(LSTMStateTuple(c=array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), h=array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)),
 LSTMStateTuple(c=array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), h=array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
  

In [32]:
embedding_vocab = tf.get_variable('embedding_vocab', [vocab_size, embedding_vector_size])
sess.run(tf.global_variables_initializer())
sess.run(embedding_vocab)

array([[-0.01897177,  0.01456418,  0.01162802, ...,  0.00333041,
        -0.01658814, -0.00616716],
       [ 0.00745418, -0.02214274,  0.00963265, ...,  0.015057  ,
         0.02205976, -0.00230152],
       [-0.00717242, -0.01427738,  0.00288171, ..., -0.02357768,
         0.01343996, -0.01721073],
       ...,
       [ 0.00698355, -0.02285169, -0.01921256, ...,  0.01569381,
        -0.00027356, -0.01569029],
       [-0.00267484, -0.01727916, -0.01470053, ..., -0.01046467,
        -0.01682112,  0.000709  ],
       [ 0.00075604, -0.00512131,  0.00041197, ...,  0.02352624,
         0.01395251, -0.02387018]], dtype=float32)